In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import gensim

In [3]:
from gensim.models import Word2Vec, KeyedVectors

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import matplotlib.pyplot as plt
import nltk

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
data = pd.read_csv("data/news_class.csv")

In [7]:
df = data.copy() # to be safe and avoid errors

In [8]:
df = df.loc[:,["data_id" , "content" , "category_level_1" , "category_level_2"]]

In [9]:
def labeler(dataframe_column):
    encoder=LabelEncoder()
    
    labels = encoder.fit_transform(dataframe_column)
    print(encoder.classes_)
    print("We did it boys , labels have been created")
    
    return(pd.DataFrame(labels))

In [10]:
# Function for removing ASCII characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

# Function for converting to lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

# Function for removing html
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def lemm_text(text):
    lemm=WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    tokens = text
    return ' '.join([lemm.lemmatize(t) for t in tokens])

In [11]:
df['Cleaned'] = df['content'].apply(_removeNonAscii)
df['Cleaned'] = df.Cleaned.apply(func = make_lower_case)
df['Cleaned'] = df.Cleaned.apply(func = remove_stop_words)
df['Cleaned'] = df.Cleaned.apply(func = remove_punctuation)
df['Cleaned'] = df.Cleaned.apply(func = remove_html)
df['Cleaned'] = df.Cleaned.apply(func = lemm_text)
df = df.drop('content' , axis =1)

In [12]:
io = df.copy()

In [13]:
io = io.drop([374]).reset_index(drop=True)
io = io.drop([6527]).reset_index(drop=True)


corpus_full = []
for words in io['Cleaned']:
    corpus_full.append(words.split())  

In [14]:
def vectorizer():
    
    
    # Creating a list for storing the vectors ('Description' into vectors)
    #global word_embeddings
    word_embeddings = []
    i = 0
    # Reading the each 'Description'
    for line in io['Cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.key_to_index:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model.wv[word]
                else:
                    avgword2vec = avgword2vec + model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)
        else:
            print("I found it , the error occurs at line:" , i)
          
        i +=1 

    return(pd.DataFrame(word_embeddings))  # Returning our Data as a Dataframe (aesthetic reasons)

In [15]:
model = Word2Vec.load("word2vec.model3")

In [16]:
vect = vectorizer()

In [17]:
y1 = labeler(io["category_level_1"])

['arts, culture, entertainment and media' 'conflict, war and peace'
 'crime, law and justice' 'disaster, accident and emergency incident'
 'economy, business and finance' 'education' 'environment' 'health'
 'human interest' 'labour' 'lifestyle and leisure' 'politics'
 'religion and belief' 'science and technology' 'society' 'sport'
 'weather']
We did it boys , labels have been created


In [1052]:
X = vect.copy()

In [1053]:
X

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.010030,-0.072825,-0.051542,0.033568,0.084110,0.027619,-0.071665,0.015907,-0.006390,0.152007,...,-0.087481,0.150550,0.010014,0.063487,0.218890,0.049933,0.014407,-0.129096,0.003949,-0.131842
1,0.071578,-0.083879,-0.080275,0.015637,0.068045,0.007964,-0.054446,0.003107,0.003455,0.160702,...,-0.032729,0.109924,0.010929,0.027677,0.182586,0.082423,-0.014971,-0.097462,-0.027377,-0.059375
2,0.028660,-0.003605,-0.043669,0.050716,0.109462,0.030837,-0.033665,0.022952,-0.026909,0.246786,...,-0.016610,0.091539,-0.029639,0.123434,0.160820,0.140771,-0.031121,-0.071247,-0.039745,-0.078560
3,0.068074,-0.112016,-0.055053,0.045375,0.063676,0.016817,-0.061693,-0.026442,-0.031120,0.191916,...,-0.073135,0.135709,0.006571,0.004580,0.166603,0.121948,-0.031010,-0.095062,0.007649,-0.106070
4,0.040953,-0.028588,-0.063608,0.046010,0.081183,-0.021608,-0.077216,0.027718,-0.030726,0.168264,...,-0.064928,0.073787,-0.000367,0.074108,0.142589,0.093344,0.004230,-0.099472,-0.069675,-0.046723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10910,0.025258,-0.056760,-0.053916,-0.080924,0.025728,-0.042500,0.016543,-0.056844,-0.068292,0.035164,...,-0.116704,0.099249,0.008260,0.035744,0.203707,-0.003986,0.023136,-0.156268,-0.018444,-0.033398
10911,0.122435,-0.154742,-0.159965,-0.032322,0.095427,-0.047862,0.000538,-0.144428,-0.148185,0.052159,...,-0.061774,0.067999,0.021077,0.097443,0.181980,0.091247,0.007482,-0.320918,-0.028273,-0.004733
10912,0.079171,-0.031492,-0.051477,-0.086577,0.101841,0.017272,-0.046363,-0.008180,-0.058952,0.015870,...,-0.013864,0.051082,-0.004890,0.170762,0.141724,0.000773,-0.022615,-0.110620,0.037668,-0.070821
10913,0.015078,-0.098757,-0.091811,-0.046112,0.102823,0.005763,-0.019206,-0.023700,-0.079233,0.044035,...,-0.014461,0.088623,0.036835,0.127098,0.156737,-0.003581,-0.047630,-0.176579,0.055320,-0.069404


# BUILDING OUR KERAS MODEL

In [20]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [1054]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2,random_state=42)

In [1055]:
X_train.shape

(8732, 200)

In [1056]:
y_train

,0
6365,14
157,2
6965,6
6350,14
1441,6
...,...
5734,10
5191,2
5390,10
860,16


In [1057]:
y_train.values

array([[14],
       [ 2],
       [ 6],
       ...,
       [10],
       [16],
       [14]])

In [1058]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
6365,0.001412,-0.089413,-0.031683,-0.037935,0.048329,0.035798,-0.091298,0.048741,0.032696,0.187209,...,-0.022593,0.096359,-0.080201,0.085936,0.189702,0.076096,-0.020128,-0.063999,0.012825,-0.098637
157,-0.037040,-0.126159,0.038250,0.026683,0.147097,-0.025424,-0.025752,-0.047796,0.082644,0.147009,...,-0.149758,0.045411,-0.025539,0.055908,0.151813,0.074059,-0.013114,-0.068665,-0.146725,-0.106927
6965,0.136765,-0.073840,-0.003237,-0.054402,0.053695,-0.120054,-0.066894,0.066666,-0.066535,0.121522,...,-0.080405,0.042612,-0.030535,-0.029383,0.188531,-0.081600,0.035540,-0.124430,-0.054410,-0.029841
6350,-0.000632,-0.079695,-0.060020,0.036223,0.069463,-0.011242,-0.001459,0.082717,-0.023628,0.157795,...,-0.112721,0.068678,-0.090521,0.032235,0.183178,0.041957,-0.022084,-0.085274,-0.075738,-0.077661
1441,0.042041,-0.027076,-0.001995,-0.019110,0.069002,-0.054939,-0.087535,0.045370,-0.058216,0.082405,...,-0.092332,0.038480,-0.027622,-0.012296,0.195847,-0.009623,-0.009901,-0.156496,-0.046219,-0.039477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.040580,-0.027789,-0.064452,0.005701,0.062619,-0.024159,-0.031394,0.098202,-0.044198,0.151025,...,-0.091051,0.033429,-0.055286,0.172773,0.174162,0.079230,-0.033152,-0.020471,-0.081181,-0.109213
5191,0.060948,-0.050229,-0.032799,0.005161,0.045406,-0.011394,-0.062449,0.019146,0.060947,0.074583,...,-0.058806,0.037037,0.007057,-0.050395,0.211013,0.047149,-0.075033,-0.164042,-0.011013,-0.026902
5390,0.015057,-0.055502,-0.063917,-0.013218,0.086386,-0.017557,-0.090962,0.049156,-0.030447,0.207109,...,-0.049640,0.091536,-0.004806,0.134578,0.176140,-0.021786,-0.012576,-0.016673,-0.012468,-0.078947
860,0.027982,0.031712,-0.046716,-0.025009,0.058137,-0.027454,-0.052175,0.028343,-0.050664,0.153243,...,-0.162377,0.050882,-0.046641,0.146590,0.154606,-0.092813,0.000621,-0.074791,-0.087474,-0.067671


In [1059]:
X_train.shape

(8732, 200)

In [1060]:
y_train1 = to_categorical(y_train)

In [1061]:
y_test1 = to_categorical(y_test)

In [1062]:
y_train1

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [1063]:
X_val = X_train[7859:]
y_val = y_train1[7859:]

## MODEL

In [31]:
in_dim = 200

In [32]:
jazz_size = 32

In [258]:
tot_layers = 600

In [259]:
(tot_layers//3)*2

400

In [260]:
model = Sequential()
model.add(Dense(tot_layers ,input_dim=in_dim,activation = "relu"))
model.add(Dense(300,activation = "relu"))
model.add(Dense(150,activation = "relu"))
model.add(Dense(80,activation = "relu"))
model.add(Dense(40,activation = "relu"))
#model.add(Dense(30,activation = "leaky_relu"))
model.add(Dense(20,activation = "relu"))
model.add(Dense(17,activation = "softmax")) 

In [261]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [262]:
model.compile(optimizer = opt , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [263]:
model.fit(X_train[:7859] , y_train1[:7859] , batch_size = 15 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10
524/524 [==============================] - 5s 5ms/step - loss: 2.7890 - accuracy: 0.9411 - precision: 0.2000 - recall: 3.8173e-04 - val_loss: 2.7686 - val_accuracy: 0.9412 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
524/524 [==============================] - 2s 5ms/step - loss: 2.7748 - accuracy: 0.9412 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.7653 - val_accuracy: 0.9412 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
524/524 [==============================] - 2s 4ms/step - loss: 2.7747 - accuracy: 0.9412 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.7697 - val_accuracy: 0.9412 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
524/524 [==============================] - 2s 4ms/step - loss: 2.7739 - accuracy: 0.9412 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.7690 - val_accuracy: 0.9412 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
524/524 [========================

In [243]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 450)               90450     
                                                                 
 dense_72 (Dense)            (None, 300)               135300    
                                                                 
 dense_73 (Dense)            (None, 180)               54180     
                                                                 
 dense_74 (Dense)            (None, 120)               21720     
                                                                 
 dense_75 (Dense)            (None, 60)                7260      
                                                                 
 dense_76 (Dense)            (None, 50)                3050      
                                                                 
 dense_77 (Dense)            (None, 40)               

In [244]:
scores = model.evaluate(X_test, y_test1)

for i, m in enumerate(model.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

69/69 [==============================] - 0s 3ms/step - loss: 2.7828 - accuracy: 0.9412 - precision: 0.0000e+00 - recall: 0.0000e+00

loss: 2.783

accuracy: 0.941

precision: 0.000

recall: 0.000


In [245]:
from sklearn.metrics import classification_report, confusion_matrix

In [246]:
predictions = np.argmax(model.predict(X_test), axis=-1)

69/69 [==============================] - 0s 3ms/step


In [247]:
predictions

array([14, 14, 14, ..., 14, 14, 14], dtype=int64)

In [248]:
len(predictions)

2183

In [249]:
y_test.shape

(2183, 1)

In [250]:
y_test.values

array([[14],
       [ 1],
       [16],
       ...,
       [11],
       [14],
       [12]])

In [251]:
print(classification_report(y_test.values , predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        68
           1       0.00      0.00      0.00       176
           2       0.00      0.00      0.00        97
           3       0.00      0.00      0.00        90
           4       0.00      0.00      0.00        93
           5       0.00      0.00      0.00       108
           6       0.00      0.00      0.00       126
           7       0.00      0.00      0.00       136
           8       0.00      0.00      0.00       122
           9       0.00      0.00      0.00       155
          10       0.00      0.00      0.00        61
          11       0.00      0.00      0.00       172
          12       0.00      0.00      0.00       182
          13       0.00      0.00      0.00       151
          14       0.09      1.00      0.17       200
          15       0.00      0.00      0.00       169
          16       0.00      0.00      0.00        77

    accuracy              

C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [252]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [253]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions, average='macro')))
print('F1:', f1_score(y_test.values, predictions, average='macro'))

Precision is 0.00538923769232842
Recall is 0.058823529411764705
F1: 0.009873861420354967


C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MODEL 2

In [367]:
from tensorflow.keras.layers import LSTM , Dropout

In [271]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [275]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(X_train, look_back)
testX, testY = create_dataset(X_test, look_back)
valX, valY = create_dataset(X_val, look_back)

In [290]:
trainY.shape

(8730,)

In [293]:
y_train.shape

(8732, 1)

In [291]:
trainY

array([0.37080368, 0.70636785, 0.44109678, ..., 0.5599871 , 0.47138622,
       0.49634132], dtype=float32)

In [326]:
trainY = to_categorical(trainY)
testY = to_categorical(testY)
valY = to_categorical(valY)

In [327]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
valX = np.reshape(valX, (valX.shape[0], 1, valX.shape[1]))

In [337]:
trainY

array([[[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       ...,

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]]], dtype=float32)

In [365]:
model2 = Sequential()
model2.add(LSTM(100 ,input_dim=in_dim, input_shape=(200) , activation = "relu"))
model2.add(Dense(120,activation = "relu"))
#model2.add(Dense(120,activation = "relu"))
model2.add(Dense(60,activation = "relu"))
model2.add(Dense(30,activation = "relu"))
model2.add(Dense(20,activation = "relu"))
model2.add(Dense(17,activation = "softmax"))

TypeError: 'int' object is not iterable

In [362]:
model2.compile(optimizer = 'sgd' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [363]:
model2.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise V

In [364]:
model2.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 100)               40800     
                                                                 
 dense_173 (Dense)           (None, 120)               12120     
                                                                 
 dense_174 (Dense)           (None, 60)                7260      
                                                                 
 dense_175 (Dense)           (None, 30)                1830      
                                                                 
 dense_176 (Dense)           (None, 20)                620       
                                                                 
 dense_177 (Dense)           (None, 17)                357       
                                                                 
Total params: 62987 (246.04 KB)
Trainable params: 629

In [350]:
scores = model2.evaluate(testX, testY)

for i, m in enumerate(model2.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

ValueError: in user code:

    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Desktop\anaconda3\Lib\site-packages\keras\src\engine\compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sa

In [318]:
predictions2 = np.argmax(model2.predict(testX), axis=-1)

69/69 [==============================] - 1s 3ms/step


In [319]:
predictions2

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [320]:
print(classification_report(y_test.values , predictions2))

ValueError: Found input variables with inconsistent numbers of samples: [2183, 2181]

In [321]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions2, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions2, average='macro')))
print('F1:', f1_score(y_test.values, predictions2, average='macro'))

ValueError: Found input variables with inconsistent numbers of samples: [2183, 2181]

## MODEL3

In [405]:
model3 = Sequential()
model3.add(Dense(180,input_dim=in_dim,activation = "relu"))
model3.add(Dropout(0.1))
model3.add(Dense(120,activation = "relu"))
model3.add(Dense(60,activation = "relu"))
model3.add(Dropout(0.3))
model3.add(Dense(30,activation = "relu"))
model3.add(Dense(20,activation = "relu"))
model3.add(Dense(17,activation = "softmax"))

In [438]:
model3.compile(optimizer = 'adam' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [463]:
model3.fit(X_train[:7859] , y_train1[:7859] , batch_size = 32 , epochs = 20 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/20
246/246 [==============================] - 1s 4ms/step - loss: 0.8579 - accuracy: 0.9694 - precision: 0.7950 - recall: 0.6470 - val_loss: 1.0736 - val_accuracy: 0.9676 - val_precision: 0.7656 - val_recall: 0.6472
Epoch 2/20
246/246 [==============================] - 1s 3ms/step - loss: 0.8665 - accuracy: 0.9691 - precision: 0.7902 - recall: 0.6461 - val_loss: 1.0562 - val_accuracy: 0.9669 - val_precision: 0.7624 - val_recall: 0.6357
Epoch 3/20
246/246 [==============================] - 1s 3ms/step - loss: 0.8507 - accuracy: 0.9693 - precision: 0.7962 - recall: 0.6417 - val_loss: 1.0500 - val_accuracy: 0.9681 - val_precision: 0.7737 - val_recall: 0.6460
Epoch 4/20
246/246 [==============================] - 1s 3ms/step - loss: 0.8532 - accuracy: 0.9691 - precision: 0.7935 - recall: 0.6424 - val_loss: 1.0327 - val_accuracy: 0.9678 - val_precision: 0.7770 - val_recall: 0.6346
Epoch 5/20
246/246 [==============================] - 1s 3ms/step - loss: 0.8619 - accuracy: 0.9697 - pr

In [464]:
scores = model3.evaluate(X_test, y_test1)

for i, m in enumerate(model3.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

69/69 [==============================] - 0s 2ms/step - loss: 1.0355 - accuracy: 0.9666 - precision: 0.7561 - recall: 0.6377

loss: 1.036

accuracy: 0.967

precision: 0.756

recall: 0.638


In [465]:
predictions3 = np.argmax(model3.predict(X_test), axis=-1)

69/69 [==============================] - 0s 2ms/step


In [466]:
predictions3

array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [467]:
print(classification_report(y_test.values , predictions3))

              precision    recall  f1-score   support

           0       0.60      0.54      0.57        68
           1       0.75      0.75      0.75       176
           2       0.67      0.76      0.71        97
           3       0.71      0.66      0.68        90
           4       0.71      0.52      0.60        93
           5       0.67      0.73      0.70       108
           6       0.78      0.86      0.82       126
           7       0.70      0.75      0.73       136
           8       0.68      0.63      0.65       122
           9       0.82      0.68      0.75       155
          10       0.59      0.70      0.64        61
          11       0.58      0.59      0.58       172
          12       0.63      0.83      0.71       182
          13       0.65      0.58      0.62       151
          14       0.65      0.47      0.55       200
          15       0.85      0.85      0.85       169
          16       0.77      0.97      0.86        77

    accuracy              

In [468]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions3, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions3, average='macro')))
print('F1:', f1_score(y_test.values, predictions3, average='macro'))

Precision is 0.693538657098652
Recall is 0.6992945754023554
F1: 0.6918869763924599


In [194]:
# the result above is for 20 epochs

## MODEL 4

In [195]:
model4 = Sequential()
model4.add(Dense(300,input_dim=in_dim,activation = "relu"))
model4.add(Dense(120,activation = "relu"))
model4.add(Dense(60,activation = "relu"))
model4.add(Dense(30,activation = "relu"))
model4.add(Dense(20,activation = "relu"))
model4.add(Dense(17,activation = "softmax"))

In [196]:
model4.compile(optimizer = 'adam' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [198]:
model4.fit(X_train[:7859] , y_train1[:7859] , batch_size = 32 , epochs = 30 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/30
246/246 [==============================] - 1s 3ms/step - loss: 1.2867 - accuracy: 0.9572 - precision: 0.7395 - recall: 0.4200 - val_loss: 1.2383 - val_accuracy: 0.9591 - val_precision: 0.7481 - val_recall: 0.4593
Epoch 2/30
246/246 [==============================] - 1s 3ms/step - loss: 1.2637 - accuracy: 0.9579 - precision: 0.7437 - recall: 0.4326 - val_loss: 1.2782 - val_accuracy: 0.9585 - val_precision: 0.7429 - val_recall: 0.4502
Epoch 3/30
246/246 [==============================] - 1s 3ms/step - loss: 1.2414 - accuracy: 0.9583 - precision: 0.7427 - recall: 0.4464 - val_loss: 1.3162 - val_accuracy: 0.9595 - val_precision: 0.7625 - val_recall: 0.4525
Epoch 4/30
246/246 [==============================] - 1s 3ms/step - loss: 1.2068 - accuracy: 0.9597 - precision: 0.7528 - recall: 0.4680 - val_loss: 1.2205 - val_accuracy: 0.9619 - val_precision: 0.7740 - val_recall: 0.4983
Epoch 5/30
246/246 [==============================] - 1s 3ms/step - loss: 1.2224 - accuracy: 0.9587 - pr

In [199]:
# 40 EPOCHS IN TOTAL

In [200]:
scores = model4.evaluate(X_test, y_test1)

for i, m in enumerate(model4.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

69/69 [==============================] - 0s 2ms/step - loss: 1.0413 - accuracy: 0.9656 - precision: 0.7975 - recall: 0.5557

loss: 1.041

accuracy: 0.966

precision: 0.798

recall: 0.556


In [201]:
predictions4 = np.argmax(model4.predict(X_test), axis=-1)

69/69 [==============================] - 0s 2ms/step


In [202]:
predictions4

array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [203]:
print(classification_report(y_test.values , predictions4))

              precision    recall  f1-score   support

           0       0.44      0.56      0.49        68
           1       0.67      0.82      0.74       176
           2       0.59      0.78      0.67        97
           3       0.80      0.59      0.68        90
           4       0.56      0.69      0.62        93
           5       0.74      0.61      0.67       108
           6       0.78      0.77      0.78       126
           7       0.76      0.67      0.71       136
           8       0.61      0.64      0.63       122
           9       0.85      0.63      0.72       155
          10       0.63      0.59      0.61        61
          11       0.58      0.56      0.57       172
          12       0.75      0.75      0.75       182
          13       0.77      0.51      0.61       151
          14       0.53      0.58      0.56       200
          15       0.81      0.88      0.85       169
          16       0.82      0.96      0.89        77

    accuracy              

In [204]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions4, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions4, average='macro')))
print('F1:', f1_score(y_test.values, predictions4, average='macro'))

Precision is 0.6889257599801273
Recall is 0.6818623383178373
F1: 0.6786670782075368


## MODEL 5

In [226]:
model5 = Sequential()
model5.add(Dense(100,input_dim=in_dim,activation = "relu"))
model5.add(Dense(320,activation = "relu"))
model5.add(Dense(180,activation = "relu"))
model5.add(Dense(100,activation = "relu"))
model5.add(Dense(60,activation = "relu"))
model5.add(Dense(30,activation = "relu"))
model5.add(Dense(17,activation = "softmax"))

In [247]:
model5.compile(optimizer = 'rmsprop' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [248]:
model5.fit(X_train[:7859] , y_train1[:7859] , batch_size = 32 , epochs = 20 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/20
246/246 [==============================] - 2s 4ms/step - loss: 1.0453 - accuracy: 0.9634 - precision: 0.7600 - recall: 0.5516 - val_loss: 1.1896 - val_accuracy: 0.9630 - val_precision: 0.7411 - val_recall: 0.5704
Epoch 2/20
246/246 [==============================] - 1s 3ms/step - loss: 1.0348 - accuracy: 0.9635 - precision: 0.7594 - recall: 0.5547 - val_loss: 1.1362 - val_accuracy: 0.9642 - val_precision: 0.7560 - val_recall: 0.5785
Epoch 3/20
246/246 [==============================] - 1s 3ms/step - loss: 1.0245 - accuracy: 0.9644 - precision: 0.7663 - recall: 0.5670 - val_loss: 1.1516 - val_accuracy: 0.9632 - val_precision: 0.7496 - val_recall: 0.5624
Epoch 4/20
246/246 [==============================] - 1s 3ms/step - loss: 1.0185 - accuracy: 0.9646 - precision: 0.7678 - recall: 0.5702 - val_loss: 1.1926 - val_accuracy: 0.9619 - val_precision: 0.7399 - val_recall: 0.5441
Epoch 5/20
246/246 [==============================] - 1s 3ms/step - loss: 1.0019 - accuracy: 0.9650 - pr

In [249]:
scores = model5.evaluate(X_test, y_test1)

for i, m in enumerate(model5.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

69/69 [==============================] - 0s 2ms/step - loss: 1.0820 - accuracy: 0.9655 - precision: 0.7605 - recall: 0.6038

loss: 1.082

accuracy: 0.966

precision: 0.761

recall: 0.604


In [250]:
predictions5 = np.argmax(model5.predict(X_test), axis=-1)

69/69 [==============================] - 0s 2ms/step


In [251]:
predictions5

array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [252]:
print(classification_report(y_test.values , predictions5))

              precision    recall  f1-score   support

           0       0.70      0.47      0.56        68
           1       0.66      0.84      0.74       176
           2       0.70      0.68      0.69        97
           3       0.79      0.61      0.69        90
           4       0.71      0.48      0.58        93
           5       0.59      0.81      0.68       108
           6       0.83      0.75      0.79       126
           7       0.78      0.65      0.71       136
           8       0.65      0.52      0.58       122
           9       0.60      0.78      0.68       155
          10       0.57      0.66      0.61        61
          11       0.71      0.49      0.58       172
          12       0.78      0.73      0.75       182
          13       0.76      0.49      0.60       151
          14       0.47      0.66      0.55       200
          15       0.82      0.89      0.86       169
          16       0.85      0.97      0.91        77

    accuracy              

In [253]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions5, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions5, average='macro')))
print('F1:', f1_score(y_test.values, predictions5, average='macro'))

Precision is 0.7043281836758996
Recall is 0.6759700902720309
F1: 0.6792792116958751


### try_error

In [254]:
predictions_shitty = model5.predict(X_test)

69/69 [==============================] - 0s 2ms/step


In [255]:
class_shitty = np.argmax(predictions_shitty,axis=1)

In [256]:
class_shitty

array([14,  1, 16, ..., 11,  9, 12], dtype=int64)

In [257]:
print(classification_report(y_test.values , class_shitty))

              precision    recall  f1-score   support

           0       0.70      0.47      0.56        68
           1       0.66      0.84      0.74       176
           2       0.70      0.68      0.69        97
           3       0.79      0.61      0.69        90
           4       0.71      0.48      0.58        93
           5       0.59      0.81      0.68       108
           6       0.83      0.75      0.79       126
           7       0.78      0.65      0.71       136
           8       0.65      0.52      0.58       122
           9       0.60      0.78      0.68       155
          10       0.57      0.66      0.61        61
          11       0.71      0.49      0.58       172
          12       0.78      0.73      0.75       182
          13       0.76      0.49      0.60       151
          14       0.47      0.66      0.55       200
          15       0.82      0.89      0.86       169
          16       0.85      0.97      0.91        77

    accuracy              

In [258]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, class_shitfuck, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, class_shitfuck, average='macro')))
print('F1:', f1_score(y_test.values, class_shitfuck, average='macro'))

Precision is 0.7043281836758996
Recall is 0.6759700902720309
F1: 0.6792792116958751


## MODEL 6

In [368]:
model6 = Sequential()
model6.add(Dense(200,input_dim=in_dim,activation = "relu"))
model6.add(Dense(180,activation = "relu"))
model6.add(Dense(120,activation = "relu"))
model6.add(Dense(180,activation = "relu"))
model6.add(Dense(120,activation = "relu"))
model6.add(Dense(180,activation = "relu"))
model6.add(Dense(120,activation = "relu"))
model6.add(Dense(60,activation = "leaky_relu"))
model6.add(Dense(17,activation = "softmax"))

In [377]:
model6.compile(optimizer = 'adam' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [378]:
model6.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_169 (Dense)           (None, 200)               40200     
                                                                 
 dense_170 (Dense)           (None, 180)               36180     
                                                                 
 dense_171 (Dense)           (None, 120)               21720     
                                                                 
 dense_172 (Dense)           (None, 180)               21780     
                                                                 
 dense_173 (Dense)           (None, 120)               21720     
                                                                 
 dense_174 (Dense)           (None, 180)               21780     
                                                                 
 dense_175 (Dense)           (None, 120)             

In [385]:
model6.fit(X_train[:7859] , y_train1[:7859] , batch_size = 32 , epochs = 18 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/18
492/492 [==============================] - 2s 4ms/step - loss: 1.1710 - accuracy: 0.9604 - precision: 0.7500 - recall: 0.4906 - val_loss: 1.3691 - val_accuracy: 0.9580 - val_precision: 0.6973 - val_recall: 0.5040
Epoch 2/18
492/492 [==============================] - 2s 3ms/step - loss: 1.0896 - accuracy: 0.9621 - precision: 0.7536 - recall: 0.5281 - val_loss: 1.2175 - val_accuracy: 0.9601 - val_precision: 0.7410 - val_recall: 0.4948
Epoch 3/18
492/492 [==============================] - 2s 3ms/step - loss: 1.0459 - accuracy: 0.9631 - precision: 0.7588 - recall: 0.5464 - val_loss: 1.1399 - val_accuracy: 0.9640 - val_precision: 0.7612 - val_recall: 0.5659
Epoch 4/18
492/492 [==============================] - 2s 3ms/step - loss: 1.0210 - accuracy: 0.9642 - precision: 0.7689 - recall: 0.5596 - val_loss: 1.1649 - val_accuracy: 0.9635 - val_precision: 0.7627 - val_recall: 0.5521
Epoch 5/18
492/492 [==============================] - 2s 3ms/step - loss: 1.0118 - accuracy: 0.9643 - pr

In [386]:
scores = model6.evaluate(X_test, y_test1)


for i, m in enumerate(model6.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

69/69 [==============================] - 0s 3ms/step - loss: 1.1564 - accuracy: 0.9629 - precision: 0.7434 - recall: 0.5639

loss: 1.156

accuracy: 0.963

precision: 0.743

recall: 0.564


In [387]:
predictions6 = model6.predict(X_test)

69/69 [==============================] - 0s 3ms/step


In [388]:
predictions6 = np.argmax(predictions6,axis=1)

In [389]:
print(classification_report(y_test.values , predictions6))

              precision    recall  f1-score   support

           0       0.53      0.31      0.39        68
           1       0.67      0.83      0.74       176
           2       0.55      0.76      0.64        97
           3       0.52      0.79      0.63        90
           4       0.73      0.43      0.54        93
           5       0.56      0.71      0.63       108
           6       0.80      0.69      0.74       126
           7       0.74      0.63      0.68       136
           8       0.73      0.40      0.52       122
           9       0.75      0.61      0.68       155
          10       0.62      0.67      0.65        61
          11       0.66      0.41      0.51       172
          12       0.67      0.82      0.74       182
          13       0.49      0.65      0.56       151
          14       0.70      0.45      0.55       200
          15       0.64      0.92      0.76       169
          16       0.90      0.90      0.90        77

    accuracy              

In [390]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions6, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions6, average='macro')))
print('F1:', f1_score(y_test.values, predictions6, average='macro'))

Precision is 0.6629182165685473
Recall is 0.6469336337317027
F1: 0.6376699424054997


### MODEL 7

In [538]:
model7 = Sequential()
model7.add(Dense(100,input_dim=in_dim,activation = "leaky_relu"))
model7.add(Dropout(0.1))
model7.add(Dense(320,activation = "leaky_relu"))
model7.add(Dense(180,activation = "leaky_relu"))
model7.add(Dense(100,activation = "leaky_relu"))
model7.add(Dense(60,activation = "leaky_relu"))
model7.add(Dense(30,activation = "leaky_relu"))
model7.add(Dense(17,activation = "softmax"))

In [539]:
model7.compile(optimizer = 'adam' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [560]:
model7.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10
123/123 [==============================] - 1s 5ms/step - loss: 0.6777 - accuracy: 0.9743 - precision: 0.8274 - recall: 0.7113 - val_loss: 1.1485 - val_accuracy: 0.9666 - val_precision: 0.7493 - val_recall: 0.6506
Epoch 2/10
123/123 [==============================] - 1s 5ms/step - loss: 0.6783 - accuracy: 0.9744 - precision: 0.8264 - recall: 0.7142 - val_loss: 1.0782 - val_accuracy: 0.9666 - val_precision: 0.7445 - val_recall: 0.6575
Epoch 3/10
123/123 [==============================] - 1s 4ms/step - loss: 0.6693 - accuracy: 0.9739 - precision: 0.8238 - recall: 0.7072 - val_loss: 1.0404 - val_accuracy: 0.9683 - val_precision: 0.7624 - val_recall: 0.6690
Epoch 4/10
123/123 [==============================] - 1s 4ms/step - loss: 0.6466 - accuracy: 0.9749 - precision: 0.8303 - recall: 0.7204 - val_loss: 1.0740 - val_accuracy: 0.9675 - val_precision: 0.7529 - val_recall: 0.6667
Epoch 5/10
123/123 [==============================] - 1s 4ms/step - loss: 0.6820 - accuracy: 0.9738 - pr

In [561]:
predictions7 = model7.predict(X_test)

69/69 [==============================] - 0s 2ms/step


In [562]:
predictions7 = np.argmax(predictions7,axis=1)

In [563]:
print(classification_report(y_test.values , predictions7))

              precision    recall  f1-score   support

           0       0.67      0.59      0.62        68
           1       0.82      0.72      0.77       176
           2       0.69      0.68      0.69        97
           3       0.66      0.68      0.67        90
           4       0.74      0.54      0.62        93
           5       0.78      0.64      0.70       108
           6       0.78      0.83      0.80       126
           7       0.81      0.60      0.69       136
           8       0.67      0.69      0.68       122
           9       0.79      0.75      0.77       155
          10       0.62      0.62      0.62        61
          11       0.51      0.76      0.61       172
          12       0.73      0.74      0.73       182
          13       0.63      0.61      0.62       151
          14       0.58      0.62      0.60       200
          15       0.84      0.86      0.85       169
          16       0.89      0.92      0.90        77

    accuracy              

In [564]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions7, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions7, average='macro')))
print('F1:', f1_score(y_test.values, predictions7, average='macro'))

Precision is 0.717422154323419
Recall is 0.6968372772048624
F1: 0.7029330342203923


In [566]:
## the above is for ~= 70 epochs

In [587]:
model7.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 10 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/10
123/123 [==============================] - 1s 5ms/step - loss: 0.4897 - accuracy: 0.9801 - precision: 0.8599 - recall: 0.7909 - val_loss: 1.2282 - val_accuracy: 0.9652 - val_precision: 0.7280 - val_recall: 0.6529
Epoch 2/10
123/123 [==============================] - 1s 4ms/step - loss: 0.4533 - accuracy: 0.9813 - precision: 0.8690 - recall: 0.8025 - val_loss: 1.2694 - val_accuracy: 0.9668 - val_precision: 0.7378 - val_recall: 0.6770
Epoch 3/10
123/123 [==============================] - 1s 4ms/step - loss: 0.4536 - accuracy: 0.9816 - precision: 0.8730 - recall: 0.8049 - val_loss: 1.2530 - val_accuracy: 0.9664 - val_precision: 0.7329 - val_recall: 0.6758
Epoch 4/10
123/123 [==============================] - 1s 4ms/step - loss: 0.4457 - accuracy: 0.9820 - precision: 0.8736 - recall: 0.8107 - val_loss: 1.2660 - val_accuracy: 0.9679 - val_precision: 0.7503 - val_recall: 0.6816
Epoch 5/10
123/123 [==============================] - 1s 4ms/step - loss: 0.4566 - accuracy: 0.9816 - pr

In [588]:
predictions7 = model7.predict(X_test)

69/69 [==============================] - 0s 2ms/step


In [589]:
predictions7 = np.argmax(predictions7,axis=1)

In [590]:
print(classification_report(y_test.values , predictions7))

              precision    recall  f1-score   support

           0       0.66      0.68      0.67        68
           1       0.79      0.73      0.76       176
           2       0.64      0.80      0.72        97
           3       0.69      0.66      0.67        90
           4       0.74      0.46      0.57        93
           5       0.70      0.73      0.71       108
           6       0.80      0.64      0.71       126
           7       0.72      0.81      0.76       136
           8       0.66      0.69      0.67       122
           9       0.74      0.70      0.72       155
          10       0.68      0.64      0.66        61
          11       0.66      0.54      0.59       172
          12       0.73      0.75      0.74       182
          13       0.51      0.72      0.59       151
          14       0.61      0.61      0.61       200
          15       0.89      0.83      0.86       169
          16       0.83      0.97      0.90        77

    accuracy              

In [586]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions7, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions7, average='macro')))
print('F1:', f1_score(y_test.values, predictions7, average='macro'))

Precision is 0.7188830928421032
Recall is 0.6994434200956823
F1: 0.7015231796089909


### MODEL 8 

In [696]:
model8 = Sequential()
model8.add(Dense(500,input_dim=in_dim,activation = "leaky_relu"))
model8.add(Dropout(0.2))
model8.add(Dense(400,activation = "leaky_relu"))
model8.add(Dropout(0.2))
model8.add(Dense(320,activation = "leaky_relu"))
model8.add(Dropout(0.2))
model8.add(Dense(180,activation = "leaky_relu"))
model8.add(Dropout(0.2))
model8.add(Dense(100,activation = "leaky_relu"))
model8.add(Dense(60,activation = "leaky_relu"))
model8.add(Dense(30,activation = "leaky_relu"))
model8.add(Dense(17,activation = "softmax"))

In [697]:
model8.compile(optimizer = 'adam' , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [708]:
model8.fit(X_train[:7859] , y_train1[:7859] , batch_size = 64 , epochs = 20 , validation_data=(X_val,y_val) ,verbose =1)

Epoch 1/20
123/123 [==============================] - 1s 9ms/step - loss: 1.0175 - accuracy: 0.9632 - precision: 0.7597 - recall: 0.5488 - val_loss: 1.0518 - val_accuracy: 0.9656 - val_precision: 0.7589 - val_recall: 0.6094
Epoch 2/20
123/123 [==============================] - 1s 8ms/step - loss: 1.0229 - accuracy: 0.9639 - precision: 0.7658 - recall: 0.5576 - val_loss: 1.0543 - val_accuracy: 0.9658 - val_precision: 0.7611 - val_recall: 0.6094
Epoch 3/20
123/123 [==============================] - 1s 8ms/step - loss: 1.0281 - accuracy: 0.9634 - precision: 0.7603 - recall: 0.5517 - val_loss: 1.0789 - val_accuracy: 0.9646 - val_precision: 0.7625 - val_recall: 0.5773
Epoch 4/20
123/123 [==============================] - 1s 8ms/step - loss: 1.0167 - accuracy: 0.9642 - precision: 0.7674 - recall: 0.5614 - val_loss: 1.0709 - val_accuracy: 0.9664 - val_precision: 0.7907 - val_recall: 0.5842
Epoch 5/20
123/123 [==============================] - 1s 9ms/step - loss: 1.0029 - accuracy: 0.9647 - pr

In [709]:
predictions8 = model8.predict(X_test)

69/69 [==============================] - 0s 3ms/step


In [710]:
predictions8 = np.argmax(predictions8,axis=1)

In [711]:
print(classification_report(y_test.values , predictions8))

              precision    recall  f1-score   support

           0       0.69      0.32      0.44        68
           1       0.69      0.84      0.75       176
           2       0.81      0.53      0.64        97
           3       0.69      0.61      0.65        90
           4       0.60      0.51      0.55        93
           5       0.64      0.82      0.72       108
           6       0.83      0.72      0.77       126
           7       0.73      0.78      0.75       136
           8       0.55      0.69      0.61       122
           9       0.70      0.77      0.73       155
          10       0.71      0.61      0.65        61
          11       0.61      0.60      0.61       172
          12       0.78      0.80      0.79       182
          13       0.64      0.54      0.59       151
          14       0.59      0.60      0.60       200
          15       0.83      0.79      0.81       169
          16       0.78      0.99      0.87        77

    accuracy              

In [712]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions8, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions8, average='macro')))
print('F1:', f1_score(y_test.values, predictions8, average='macro'))

Precision is 0.6978324524777793
Recall is 0.6772860300983213
F1: 0.6783898482524271


In [695]:
# for 30 epochs model8 scored 71% accuracy , with adamW optimizer , with act_funct = leaky_relu

In [624]:
# for 30+20 epochs model8 scored 72% accuracy , with adamW optimizer , with act_funct = leaky_relu

### MODE: L 9

In [1421]:
model9 = Sequential()
model9.add(Dense(17,input_dim=in_dim,activation = "leaky_relu"))
model9.add(Dense(320,activation = "leaky_relu"))
model9.add(Dropout(0.2))
model9.add(Dense(180,activation = "leaky_relu"))
#model9.add(Dropout(0.1))
model9.add(Dense(100,activation = "leaky_relu"))
model9.add(Dense(60,activation = "leaky_relu"))
model9.add(Dense(30,activation = "leaky_relu"))
model9.add(Dense(17,activation = "softmax"))

In [1425]:
model9.compile(optimizer = opt6 , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [1426]:
model9.fit(X_train[:7858].values , y_train1[:7858] , batch_size = 64 , epochs = 55, validation_data=(X_val.values,y_val) ,verbose =1)

Epoch 1/55
123/123 [==============================] - 3s 7ms/step - loss: 1.9848 - accuracy: 0.9454 - precision: 0.6509 - recall: 0.1542 - val_loss: 1.5788 - val_accuracy: 0.9499 - val_precision: 0.6826 - val_recall: 0.2784
Epoch 2/55
123/123 [==============================] - 1s 4ms/step - loss: 1.5294 - accuracy: 0.9507 - precision: 0.6856 - recall: 0.2994 - val_loss: 1.5151 - val_accuracy: 0.9543 - val_precision: 0.7053 - val_recall: 0.3837
Epoch 3/55
123/123 [==============================] - 1s 4ms/step - loss: 1.4089 - accuracy: 0.9544 - precision: 0.7192 - recall: 0.3683 - val_loss: 1.3971 - val_accuracy: 0.9570 - val_precision: 0.7474 - val_recall: 0.4066
Epoch 4/55
123/123 [==============================] - 1s 4ms/step - loss: 1.3345 - accuracy: 0.9565 - precision: 0.7292 - recall: 0.4132 - val_loss: 1.3111 - val_accuracy: 0.9590 - val_precision: 0.7449 - val_recall: 0.4616
Epoch 5/55
123/123 [==============================] - 1s 4ms/step - loss: 1.2513 - accuracy: 0.9598 - pr

In [1427]:
predictions9 = model9.predict(X_test.values)

69/69 [==============================] - 0s 3ms/step


In [1428]:
predictions9 = np.argmax(predictions9,axis=1)

In [1429]:
print(classification_report(y_test.values , predictions9))

              precision    recall  f1-score   support

           0       0.61      0.54      0.57        68
           1       0.82      0.80      0.81       176
           2       0.74      0.72      0.73        97
           3       0.71      0.64      0.67        90
           4       0.63      0.56      0.59        93
           5       0.67      0.75      0.71       108
           6       0.85      0.81      0.83       126
           7       0.69      0.76      0.73       136
           8       0.78      0.71      0.74       122
           9       0.81      0.73      0.77       155
          10       0.68      0.66      0.67        61
          11       0.64      0.63      0.64       172
          12       0.72      0.83      0.77       182
          13       0.71      0.63      0.67       151
          14       0.56      0.59      0.58       200
          15       0.82      0.85      0.84       169
          16       0.84      0.97      0.90        77

    accuracy              

In [1430]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions9, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions9, average='macro')))
print('F1:', f1_score(y_test.values, predictions9, average='macro'))

Precision is 0.7220791807689374
Recall is 0.7176792722406762
F1: 0.7183660390237607


In [1424]:
opt6 = tf.keras.optimizers.AdamW(learning_rate =0.01 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [1325]:
# with opt3 model9 scored for 15 epochs 72% accuracy with batchsize 64

In [1410]:
model9.summary()

Model: "sequential_73"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_468 (Dense)           (None, 17)                3417      
                                                                 
 dense_469 (Dense)           (None, 320)               5760      
                                                                 
 dropout_54 (Dropout)        (None, 320)               0         
                                                                 
 dense_470 (Dense)           (None, 180)               57780     
                                                                 
 dense_471 (Dense)           (None, 100)               18100     
                                                                 
 dense_472 (Dense)           (None, 60)                6060      
                                                                 
 dense_473 (Dense)           (None, 30)              

In [1332]:
# with opt3 model9 scored for 55 epochs 73% accuracy with batchsize 64

In [1393]:
opt4 = tf.keras.optimizers.AdamW(learning_rate =0.06 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

### MODEL 10

In [1574]:
model10 = Sequential()
model10.add(Dense(17,input_dim=in_dim,activation = "leaky_relu"))
model10.add(Dense(320,activation = "leaky_relu"))
model10.add(Dropout(0.2))
#model9.add(Dense(180,activation = "leaky_relu"))
#model9.add(Dropout(0.1))
model10.add(Dense(100,activation = "leaky_relu"))
model10.add(Dropout(0.1))
#model10.add(Dense(60,activation = "leaky_relu"))
#model10.add(Dense(30,activation = "leaky_relu"))
model10.add(Dense(17,activation = "softmax"))

In [1575]:
model10.compile(optimizer = opt7 , 
              loss = 'categorical_crossentropy' ,
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

In [1576]:
opt7 = tf.keras.optimizers.AdamW(learning_rate =0.01 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [1597]:
model10.fit(X_train[:7858].values , y_train1[:7858] , batch_size = 64 , epochs = 5, validation_data=(X_val.values,y_val) ,verbose =1)

Epoch 1/5
123/123 [==============================] - 0s 4ms/step - loss: 0.8257 - accuracy: 0.9706 - precision: 0.8024 - recall: 0.6629 - val_loss: 1.2321 - val_accuracy: 0.9647 - val_precision: 0.7447 - val_recall: 0.6082
Epoch 2/5
123/123 [==============================] - 0s 3ms/step - loss: 0.8383 - accuracy: 0.9701 - precision: 0.7951 - recall: 0.6616 - val_loss: 1.1178 - val_accuracy: 0.9657 - val_precision: 0.7615 - val_recall: 0.6071
Epoch 3/5
123/123 [==============================] - 0s 3ms/step - loss: 0.8329 - accuracy: 0.9703 - precision: 0.7978 - recall: 0.6624 - val_loss: 1.1771 - val_accuracy: 0.9647 - val_precision: 0.7468 - val_recall: 0.6048
Epoch 4/5
123/123 [==============================] - 0s 3ms/step - loss: 0.8364 - accuracy: 0.9702 - precision: 0.7988 - recall: 0.6593 - val_loss: 1.2063 - val_accuracy: 0.9660 - val_precision: 0.7563 - val_recall: 0.6220
Epoch 5/5
123/123 [==============================] - 0s 3ms/step - loss: 0.8427 - accuracy: 0.9706 - precisi

In [1598]:
predictions10 = model10.predict(X_test.values)
predictions10 = np.argmax(predictions10,axis=1)

69/69 [==============================] - 0s 2ms/step


In [1599]:
print(classification_report(y_test.values , predictions10))

              precision    recall  f1-score   support

           0       0.58      0.51      0.55        68
           1       0.80      0.80      0.80       176
           2       0.72      0.80      0.76        97
           3       0.73      0.66      0.69        90
           4       0.67      0.58      0.62        93
           5       0.67      0.81      0.73       108
           6       0.85      0.83      0.84       126
           7       0.72      0.82      0.77       136
           8       0.73      0.66      0.69       122
           9       0.79      0.70      0.74       155
          10       0.70      0.64      0.67        61
          11       0.62      0.65      0.64       172
          12       0.76      0.81      0.79       182
          13       0.73      0.62      0.67       151
          14       0.61      0.60      0.61       200
          15       0.81      0.88      0.84       169
          16       0.86      0.96      0.91        77

    accuracy              

In [1600]:
    # Evaluating the model
print('Precision is {}'.format(precision_score(y_test.values, predictions10, average='macro')))
print('Recall is {}'.format(recall_score(y_test.values, predictions10, average='macro')))
print('F1:', f1_score(y_test.values, predictions10, average='macro'))

Precision is 0.7264048178436643
Recall is 0.72420541870145
F1: 0.7232623071801931


In [1606]:
score = model10.evaluate(X_test.values , y_test1 , batch_size = 32 , verbose =2)

69/69 - 0s - loss: 0.9691 - accuracy: 0.9697 - precision: 0.7916 - recall: 0.6578 - 175ms/epoch - 3ms/step
